In [1]:
from ollama import Client
from pdfquery import PDFQuery
import pdfplumber
import pandas as pd
from pprint import pprint

In [4]:


ollama_url = 'http://localhost:11434'
#model_name = 'llama3.1:8b'
model_name = 'gemma3:4b'

# Get an ollama client
llmclient = Client(host=ollama_url)

model_options = {
    'num_predict': 500,  # max number of tokens to predict
    'temperature': 0.1,
    'top_p': 0.9,
}

file_name = r"G:\Mon Drive\DDM\BidMgt\input_data\AO-2518-27052025-05-02_14-27-391.pdf"

pdf = PDFQuery(file_name)
pdf.load()

# Use CSS-like selectors to locate the elements
text_elements = pdf.pq('LTTextLineHorizontal')

# Extract the text from the elements
text = [t.text for t in text_elements if t.text != '']

print(text)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


['Mission de coordination SPS en phase conception et réalisation pour ', 'des opérations de catégorie 2, dans le cadre du programme de ', 'reconstruction des barrages de Barbey et La Brosse. ', 'Département(s) de publication : 77, 89, 10, 51, 02, 60, 95, 91, 45 ', 'Annonce n° 25-45851 ', "Nom complet de l'acheteur : VOIES NAVIGABLES DE FRANCE ", "Type de Numéro national d'indentification : SIRET ", "N° National d'identification : 13001779101412 ", 'Ville : Dijon ', 'Code postal : 21000 ', 'Groupement de commandes : Non ', 'Département(s) de publication : 77, 89, 10, 51, 02, 60, 95, 91, 45 ', 'Section 2 - Communication ', 'Identifiant interne de la consultation : SPS_BARBEY_LAB ', "Intégralité des documents sur le profil d'acheteur : Oui ", 'Utilisation de moyens de communication non communément disponibles : Non ', 'Nom du contact : Pôle Commande Publique ', 'Adresse mail du contact : Pcp.dt.centrebourgogne@vnf.fr ', 'Section 3 - Procédure ', "Conditions de participation : - Aptitude à

In [5]:
prompt_def = "Definition: A named entity is either a name that refers to an entity like a person (PERSON), a location (LOC), an organization (ORG),  an event (EVENT), a mission (MISSION) or a value expersionn denoting a date (DATE), a duration (DURATION), an ID (ID) or an amount of money (AMOUNT) \n"
prompt_task = "Task: extract all named entities mentioned from the following text and classify them into the categories defined above. \n and retrun the result in a JSON format with the following keys: 'entities' and 'text'. \n"

prompt_full = prompt_def + prompt_task + "".join(text)
result = llmclient.generate(model=model_name, prompt=prompt_full, options=model_options)

pprint(result, compact=False, sort_dicts=False)

print("LLM response:\n", result['response'])

GenerateResponse(model='gemma3:4b', created_at='2025-06-05T09:24:13.2422879Z', done=True, done_reason='length', total_duration=81059063800, load_duration=6460445600, prompt_eval_count=826, prompt_eval_duration=26192281300, eval_count=500, eval_duration=48395432900, response='```json\n{\n  "entities": [\n    {\n      "entity": "SPS",\n      "category": "ORG"\n    },\n    {\n      "entity": "Barbey",\n      "category": "LOC"\n    },\n    {\n      "entity": "La Brosse",\n      "category": "LOC"\n    },\n    {\n      "entity": "77",\n      "category": "ID"\n    },\n    {\n      "entity": "89",\n      "category": "ID"\n    },\n    {\n      "entity": "10",\n      "category": "ID"\n    },\n    {\n      "entity": "51",\n      "category": "ID"\n    },\n    {\n      "entity": "02",\n      "category": "ID"\n    },\n    {\n      "entity": "60",\n      "category": "ID"\n    },\n    {\n      "entity": "95",\n      "category": "ID"\n    },\n    {\n      "entity": "91",\n      "category": "ID"\n    },

In [ ]:
prompt_def = """Definition: a named entity is either an 'Appel offres', 'Info Acheteur' or 'Scope'. Each are structured enity as defined below with examples

Examples:
'Appel offres':
	'Identifiant avis':  '25-45851'
	'Zone application":  '77, 89, 10, 51, 02, 60, 95, 91, 45'
	'ID interne": 'SPS_BARBEY_LAB'
	'Date limite depot":  '17/05/2025 à 17:00'
	'Possibilité d'attribution sans négociation' : 'Oui'

'Info Acheteur':
	'Nom': 'VOIES NAVIGABLES DE FRANCE'
	'type ID': 'SIRET'
	'ID': '13001779101412'
	'City': 'Dijon, 2100'
	'nom contact': 'Pôle Commande Publique'
	'mail contact': 'Pcp.dt.centrebourgogne@vnf.fr'

'Scope':
	Nom: Mission de coordination SPS en phase conception et réalisation pour des opérations de catégorie 2, dans le  cadre du programme de reconstruction des barrages de Barbey et La Brosse
	Code CPV: 71317210
	Lieu Execution: Sites des barrages de Barbey et de Labrosse
	durée (mois): 84
	plusieurs tranches: non
	plusieurs lots: non
	visite obligatoire: non
	Autres infos:  Après examen des offres, le pouvoir adjudicateur engagera des négociations avec tous les  candidats sélectionnés. Toutefois, le pouvoir adjudicateur se réserve la possibilité d'attribuer le marché sur la base des offres initiales, sans négociation. 
 Le pouvoir adjudicateur se réserve la possibilité de confier ultérieurement au titulaire du marché, en application des articles L. 2122 1 et R. 2122-7 du Code de la commande publique, un ou plusieurs nouveaux marchés ayant pour objet la réalisation de prestations  similaires.

Extract all named entities mentioned in the document attached. The result should be a structed output as defined with the examples."""

prompt_full = prompt_def + "".join(text)
result = llmclient.generate(model=model_name, prompt=prompt_full, options=model_options)

pprint(result, compact=False, sort_dicts=False)

print("LLM response:\n", result['response'])

GenerateResponse(model='gemma3:4b', created_at='2025-06-03T09:44:27.2657815Z', done=True, done_reason='stop', total_duration=65713374200, load_duration=60716300, prompt_eval_count=1232, prompt_eval_duration=24827293800, eval_count=461, eval_duration=40824794700, response="Here's the structured extraction of named entities from the provided document, categorized as 'Appel offres', 'Info Acheteur', and 'Scope':\n\n**Appel offres:**\n\n*   'Identifiant avis': '25-45851'\n*   'Date limite depot': '17/05/2025 à 17:00'\n*   'Possibilité d'attribution sans négociation' : 'Oui'\n\n**Info Acheteur:**\n\n*   'Nom': 'VOIES NAVIGABLES DE FRANCE'\n*   'type ID': 'SIRET'\n*   'ID': '13001779101412'\n*   'City': 'Dijon, 2100'\n*   'nom contact': 'Pôle Commande Publique'\n*   'mail contact': 'Pcp.dt.centrebourgogne@vnf.fr'\n\n**Scope:**\n\n*   'Nom': 'Mission de coordination SPS en phase conception et réalisation pour des opérations de catégorie 2, dans le cadre du programme de reconstruction des barr

In [ ]:
file_name = r"G:\Mon Drive\DDM\BidMgt\input_data\ReglementConsultation.pdf"

pdf = PDFQuery(file_name,)
pdf.load()
text_elements = pdf.pq('LTTextLineHorizontal')

# Extract the text from the elements
text = [t.text for t in text_elements if t.text != '']
prompt_full = prompt_def + "".join(text)
result = llmclient.generate(model=model_name, prompt=prompt_full, options=model_options)

pprint(result, compact=False, sort_dicts=False)

print("LLM response:\n", result['response'])

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


GenerateResponse(model='gemma3:4b', created_at='2025-06-03T09:46:38.2386006Z', done=True, done_reason='length', total_duration=128394876700, load_duration=58481800, prompt_eval_count=4096, prompt_eval_duration=85400436200, eval_count=500, eval_duration=42935958700, response='Okay, here\'s a breakdown of the key information from this extensive document, categorized for clarity:\n\n**1. Overview & Purpose:**\n\n*   **Procurement:** This document outlines the requirements for a procurement process, likely for a construction, maintenance, or service contract related to infrastructure or public works.\n*   **Goal:** To select a qualified operator (company/bidder) to fulfill the specified requirements.\n\n**2. Submission Requirements & Process:**\n\n*   **Deadlines:** Crucially, there are strict deadlines for submitting bids:\n    *   **Bid Submission:**  (Specific date not provided in the text, but it’s a key deadline)\n    *   **Questions/Clarifications:**  Submit questions 10 days before 

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM, pipeline, AutoModelForTokenClassification

camembertav2 = AutoModel.from_pretrained("almanach/camembertav2-base")
tokenizer = AutoTokenizer.from_pretrained("almanach/camembertav2-base")

model_for_ner = AutoModelForTokenClassification.from_pretrained("almanach/camembertav2-base")

# Create a NER pipeline using the token classification model
ner_pipeline = pipeline("token-classification", model=model_for_ner, tokenizer=tokenizer, aggregation_strategy="simple")

prompt_def = "Definition: A named entity is either a name that refers to an entity like a 'Critère' (Valeur technique, valeur environnementale, valeur financière), 'libellé des sous-critèeres', des 'coefficient' (int), 'Note Pondérée obtenue', 'Note pondérée attributaire' , 'motifs'  \n"

prompt_task = "Task: extract all named entities mentioned from the following text and classify them into the categories defined above. \n and retrun the result in a JSON format with the following keys: 'entities' and 'text'. \n"

prompt_full = prompt_def + prompt_task + "".join(text)

# Extract Named Entities from the text
entities = ner_pipeline(prompt_full)

# Print the extracted entities
print(entities)

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at almanach/camembertav2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'LABEL_0', 'score': np.float32(0.57387894), 'word': 'Definition', 'start': 0, 'end': 10}, {'entity_group': 'LABEL_1', 'score': np.float32(0.50135154), 'word': ':', 'start': 10, 'end': 11}, {'entity_group': 'LABEL_0', 'score': np.float32(0.5830087), 'word': 'A', 'start': 12, 'end': 13}, {'entity_group': 'LABEL_1', 'score': np.float32(0.5092035), 'word': 'name', 'start': 14, 'end': 18}, {'entity_group': 'LABEL_0', 'score': np.float32(0.59105456), 'word': '##d entity is either a', 'start': 18, 'end': 38}, {'entity_group': 'LABEL_1', 'score': np.float32(0.51246357), 'word': 'name', 'start': 39, 'end': 43}, {'entity_group': 'LABEL_0', 'score': np.float32(0.5421785), 'word': 'that refers to an', 'start': 44, 'end': 61}, {'entity_group': 'LABEL_1', 'score': np.float32(0.5033139), 'word': 'ent', 'start': 62, 'end': 65}, {'entity_group': 'LABEL_0', 'score': np.float32(0.55361515), 'word': "##ity like a ' Critère ' ( Valeur technique, valeur", 'start': 65, 'end': 111}, {'entity

In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_core.documents import Document

file_rejet = r"C:\Users\jch_m\DocumentsPerso\Centrale-IAC\StageATAE\Rejet_ATAE.pdf"

pdf_tab = PDFPlumberLoader(file_rejet)
doc_data = pdf_tab.load()

print(doc_data)

total_pages = doc_data[0].metadata['total_pages']
print(f"Total Pages: {total_pages}")

for page_num in range(total_pages):
    page_content = doc_data[page_num].page_content
    print(page_content) # Print only a portion to keep output concise


ValueError: File path C:\Users\jch_m\DocumentsPerso\Centrale-IAC\StageATAE\Rejet_ATAE.pdf is not a valid file or url